In [47]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import shap
import matplotlib.pyplot as plt


In [48]:
data = pd.read_csv("data/yield_climate_final.csv")

In [49]:
data

,country,crop,year,yield_kg_ha,t2m,rad,rh2m,precip,lag_yield_kg_ha,crop_encoded,country_encoded
0,South Africa,Wheat,2002,2590.6,17.874615,22.388462,41.238462,8.14,2570.8,3,85
1,Spain,Maize (corn),2002,9514.2,13.202308,16.440769,64.092308,16.55,9720.8,0,86
2,Somalia,Wheat,2002,373.6,28.020000,22.956923,54.958462,9.11,365.4,3,84
3,South Africa,Maize (corn),2002,2851.6,17.874615,22.388462,41.238462,8.14,2437.1,0,85
4,South Africa,Rice,2002,2356.3,17.874615,22.388462,41.238462,8.14,2285.7,1,85
...,...,...,...,...,...,...,...,...,...,...,...
7932,Sri Lanka,Sugar cane,2023,50682.0,26.443077,18.701538,83.175385,83.57,57011.1,2,87
7933,Zimbabwe,Wheat,2023,4553.8,21.551538,21.309231,54.147692,26.88,5154.2,3,102
7934,"China, mainland",Rice,2023,7136.8,7.953077,16.070000,60.429231,16.66,7079.6,1,22
7935,"China, mainland",Wheat,2023,5781.1,7.953077,16.070000,60.429231,16.66,5855.4,3,22


In [50]:
# Sort for safety
data = data.sort_values("year")

# Time-based split
train = data[data['year'] <= 2016]
test = data[(data['year'] > 2016) & (data['year'] < 2023)]
testF = data[data['year'] == 2023] #testFuture

features = ['t2m', 'precip', 'rad', 'rh2m', 'lag_yield_kg_ha',
            'crop_encoded', 'country_encoded']

target = 'yield_kg_ha'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_F = testF[features]
y_F = testF[target]

In [63]:
features_no_lag = ['t2m','precip','rad','rh2m','crop_encoded','country_encoded']

#-----------------LINEAR REGRESSION without lag-yield -----------------------------

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train[features_no_lag], train[target])
pred = lr.predict(test[features_no_lag])

print("rmse: ", mean_squared_error(test[target], pred)**0.5)
print("Linear Regression R2 without lag =", r2_score(test[target], pred))


rmse:  25779.27179341047
Linear Regression R2 without lag = 0.09458110555508081


In [52]:
# --------------------------LightGBM without lag-yield------------------------
lgbmNL = LGBMRegressor(
    n_estimators=700,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgbmNL.fit(train[features_no_lag], train[target])

y_pred_lgbm = lgbmNL.predict(test[features_no_lag])
mse_lgbm = mean_squared_error(test[target], y_pred_lgbm)
rmse_lgbm = mse_lgbm ** 0.5
r2_lgbm = r2_score(test[target], y_pred_lgbm)

print("LightGBM  RMSE:", rmse_lgbm)
print("LightGBM  R2  :", r2_lgbm)

# --------------------------XGBoost without lag-yield------------------------
xgbNL = XGBRegressor(
    n_estimators=700,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    max_depth=6,
    random_state=42,
    tree_method="hist"  
)

xgbNL.fit(train[features_no_lag], train[target])

y_pred_xgb = xgbNL.predict(test[features_no_lag])
mse_xgb = mean_squared_error(test[target], y_pred_lgbm)
rmse_xgb = mse_xgb ** 0.5
r2_xgb = r2_score(test[target], y_pred_lgbm)

print("XGB  RMSE:", rmse_lgbm)
print("XGB  R2  :", r2_lgbm)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000382 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 5423, number of used features: 6
[LightGBM] [Info] Start training from score 16554.123830
LightGBM  RMSE: 5517.506326987455
LightGBM  R2  : 0.9585242792105777
XGB  RMSE: 5517.506326987455
XGB  R2  : 0.9585242792105777


In [64]:
# ----------------- LightGBM -----------------
lgbm = LGBMRegressor(
    n_estimators=700,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
lr = LinearRegression()
lgbm.fit(X_train, y_train)

y_pred_lgbm = lgbm.predict(X_test)
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
rmse_lgbm = mse_lgbm ** 0.5
r2_lgbm = r2_score(y_test, y_pred_lgbm)

print("\nLightGBM  RMSE          :", rmse_lgbm)
print("LightGBM  R2            :", r2_lgbm)

#for 2023
pred2023 = lgbm.predict(X_F)
print("\nLightGBM  RMSE for 2023 :", mean_squared_error(y_F,pred2023)**0.5)
print("LightGBM R2 for 2023    :", r2_score(y_F, pred2023))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1382
[LightGBM] [Info] Number of data points in the train set: 5423, number of used features: 7
[LightGBM] [Info] Start training from score 16554.123830

LightGBM  RMSE          : 3126.352459177482
LightGBM  R2            : 0.9866836825909822

LightGBM  RMSE for 2023 : 2847.710300815933
LightGBM R2 for 2023    : 0.9890256308280415


In [65]:
# ----------------- XGBoost -----------------
xgb = XGBRegressor(
    n_estimators=700,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    max_depth=6,
    random_state=42,
    tree_method="hist"   # fast
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = mse_xgb ** 0.5
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost  RMSE          :", rmse_lgbm)
print("XGBoost  R2            :", r2_lgbm)

#for 2023
pred2023 = xgb.predict(X_F)
print("\nXGBoost  RMSE for 2023 :", mean_squared_error(y_F,pred2023)**0.5)
print("XGBoost R2 for 2023    :", r2_score(y_F, pred2023))

XGBoost  RMSE          : 3126.352459177482
XGBoost  R2            : 0.9866836825909822

XGBoost  RMSE for 2023 : 2620.8141955202623
XGBoost R2 for 2023    : 0.9907047640575919


In [55]:
if r2_lgbm >= r2_xgb:
    best_model = lgbm
    best_name = "LightGBM"
else:
    best_model = xgb
    best_name = "XGBoost"

print(f"\nUsing {best_name} as the main model for SHAP and interpretation.")



Using LightGBM as the main model for SHAP and interpretation.


In [56]:
#rechecking order
feature_names = ['t2m', 'precip', 'rad', 'rh2m',
                 'lag_yield_kg_ha', 'crop_encoded', 'country_encoded']

# Rebuild X_test (just to be safe)
X_test = X_test[feature_names]

# Take a subsample of test data for SHAP
X_shap = X_test.copy()
if len(X_shap) > 1000:
    X_shap = X_shap.sample(1000, random_state=12)

print("SHAP sample size:", len(X_shap))

# 1. Build TreeExplainer and compute SHAP values ----
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_shap)

# Convert to DataFrame for nicer handling
X_shap_df = pd.DataFrame(X_shap, columns=feature_names)

# 2. Global feature importance (summary plot) ----
plt.figure()
shap.summary_plot(shap_values, X_shap_df, show=False)
plt.tight_layout()
plt.savefig("figures/shap_summary_global.png", dpi=300)
plt.close()
print("Saved: shap_summary_global.png")

# 3. Dependence plot: precipitation vs yield effect ----
plt.figure()
shap.dependence_plot('precip', shap_values, X_shap_df, show=False)
plt.tight_layout()
plt.savefig("figures/shap_dependence_precip.png", dpi=300)
plt.close()
print("Saved: shap_dependence_precip.png")

# ---- 4. Dependence plot: temperature vs yield effect ----
plt.figure()
shap.dependence_plot('t2m', shap_values, X_shap_df, show=False)
plt.tight_layout()
plt.savefig("figures/shap_dependence_t2m.png", dpi=300)
plt.close()
print("Saved: shap_dependence_t2m.png")

# ---- 5. Dependence plot: lag_yield (very important feature) ----
plt.figure()
shap.dependence_plot('lag_yield_kg_ha', shap_values, X_shap_df, show=False)
plt.tight_layout()
plt.savefig("figures/shap_dependence_lag_yield.png", dpi=300)
plt.close()
print("Saved: shap_dependence_lag_yield.png")


SHAP sample size: 1000
Saved: shap_summary_global.png
Saved: shap_dependence_precip.png
Saved: shap_dependence_t2m.png
Saved: shap_dependence_lag_yield.png


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [61]:

# shap_values is (n_samples, n_features)
importance = np.mean(np.abs(shap_values), axis=0)

feature_importance = pd.DataFrame({
    "feature": feature_names,
    "mean_abs_shap": importance
}).sort_values("mean_abs_shap", ascending=False)

print(feature_importance)


           feature  mean_abs_shap
4  lag_yield_kg_ha   16854.524712
5     crop_encoded    3663.188558
2              rad     435.047870
6  country_encoded     340.295318
0              t2m     294.129820
1           precip     291.440173
3             rh2m     219.224955


In [62]:
normalized = importance / importance.sum()

feature_importance = pd.DataFrame({
    "feature": feature_names,
    "normalized_importance": normalized
}).sort_values("normalized_importance", ascending=False)


print(feature_importance)

           feature  normalized_importance
4  lag_yield_kg_ha               0.762722
5     crop_encoded               0.165771
2              rad               0.019687
6  country_encoded               0.015399
0              t2m               0.013310
1           precip               0.013189
3             rh2m               0.009921
